In [ ]:
import torch

### 0，Gpu的调用

In [ ]:
# 设置设备为 GPU 或 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# tensor.to(device):将张量从一个设备移动到另一个设备。device可以是'cuda'（GPU）或'cpu'，也可以指定具体的GPU，如'cuda:0'表示第一个 GPU。
# model.to(device):将模型的所有参数转移到指定设备。

# 创建一个张量并将其移动到 GPU（如果可用）
x = torch.randn(3, 3)
x = x.to(device)

# 创建模型并将其移动到 GPU（如果可用）
model = torch.nn.Linear(3, 3)
model = model.to(device)



### 1, torchvision：专门用于计算机视觉任务的拓展库

#### （1），图像数据集模块：datasets

In [ ]:
from torchvision import datasets

# 1，提供常用的图像分类数据集

train_dataset = datasets.FashionMNIST(
    root="data", # 在当前目录保存在data文件夹中
    train=True,  # 加载训练集数据（False，则加载测试集数据）
    download=True, # 下载数据
    transform=transform # 加载数据时，将每个图像转换为PyTorch张量
)

val_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

# 2，用于从目录结构中加载图像数据集（其中每个文件夹表示一个类别，文件夹中包含该类别的图像文件）

# 加载训练数据集
train_dataset = datasets.ImageFolder(root='path_to_train_data', transform=transform)
# 加载验证数据集
val_dataset = datasets.ImageFolder(root='path_to_val_data', transform=transform)

#### （2），图像变换、预处理模块：transforms

In [ ]:
from torchvision import transforms

# 提供了许多常见的图像变换操作，帮助进行数据预处理

transform = transforms.Compose([
    transforms.Resize((96, 96)),  # 调整大小为96*96
    transforms.RandomCrop((64, 64)),  # 随机裁剪为64*64的区域
    transforms.RandomHorizontalFlip(p=0.5),  # 以 50% 的概率随机水平翻转图像
    transforms.RandomRotation(45),  # 随机旋转,-45度到45度
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
])

### 2，utils：工具模块，提供了一些实用工具来帮助处理数据加载、可视化、模型训练、并行化等常见任务

#### （1），数据处理模块：data

In [ ]:
from torch.utils.data import Dataset
# 自定义数据集类，用于封装你的数据：Dataset

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data # 输入数据，比如图像
        self.labels = labels

    def __len__(self):
        """返回数据集的大小"""
        return len(self.data)

    def __getitem__(self, idx):
        """根据索引返回一个样本"""
        data = self.data[idx]
        label = self.labels[idx]
        return data, label


from torch.utils.data import DataLoader
# 批量加载数据，并支持多线程加载、数据打乱等功能，与Dataset配合使用：DataLoader

# 创建 DataLoader 对象
train_dataset = MNISTDataset(X_train, y_train, transform=transform)
val_dataset = MNISTDataset(X_val, y_val, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True) # shuffle参数:是否在每个epoch后打乱数据
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)


from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
# 从一个现有的数据集中提取一个子集，通常配合索引使用（可以用于划分图像文件训练集和测试集）

full_dataset = datasets.ImageFolder(root='data/train', transform=transform)

# 划分训练集和验证集的索引
indices = list(range(len(full_dataset)))
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)


#### （2），可视化模块：tensorboard

In [ ]:
from torch.utils.tensorboard import SummaryWriter
# 见tensorboard专题文件

### 3，nn模块：构建和训练神经网络的核心部分。它提供了许多用于定义神经网络层、损失函数等的工具

#### （1），神经网络层（Layers）

In [ ]:
from torch import nn

layer = nn.Linear(in_features=10, out_features=5) # 线性层 （全连接层） # flatten = nn.Flatten()将高维张量展平为二维张量，用于将卷积层的输出传递给全连接层。

conv = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=1)# 2d卷积层 （参数：表示3个输入通道（如RGB图像），输出16个通道，卷积核大小为3x3，步长和填充）

pool = nn.MaxPool2d(kernel_size=2, stride=2) # 2d最大池化层 （参数：表示使用2x2的窗口进行池化操作，步长为2） （nn.AvgPool1d：1d平均池化层，d表示维度）

bn = nn.BatchNorm2d(num_features=16) # 2d批量归一化层 （参数：指定了输入张量中的通道数）

rnn = nn.RNN(input_size=10, hidden_size=20, num_layers=2) # 循环神经网络层

lstm = nn.LSTM(input_size=10, hidden_size=20, num_layers=2) # 长短期记忆（LSTM）层

gru = nn.GRU(input_size=10, hidden_size=20, num_layers=2) # 门控循环单元（GRU）层

embedding = nn.Embedding(num_embeddings=1000, embedding_dim=128) # 嵌入层 （参数：词汇表大小，每个词汇嵌入向量的维度）

attention = nn.MultiheadAttention(embed_dim=256, num_heads=8) # 多头自注意力层

dropout = nn.Dropout(p=0.5) # Dropout层，用于随机丢弃神经网络中的部分神经元，以减少过拟合。


#### （2），激活函数（Activation Functions）

In [ ]:
from torch import nn

relu = nn.ReLU()

sigmoid = nn.Sigmoid()

softmax = nn.Softmax()

tanh = nn.Tanh()

prelu = nn.PReLU()


#### （3），损失函数（Loss Functions）

In [ ]:
from torch import nn

mse_loss = nn.MSELoss() # 均方误差损失函数

cross_entropy_loss = nn.CrossEntropyLoss() # 交叉熵损失函数

#### （4），模块（module）

In [ ]:
from torch import nn
"""
nn.Module 是所有神经网络模块的基类。所有的神经网络层（如 nn.Linear）都是从 nn.Module 继承的。
要创建一个自定义的神经网络模型，通常会继承 nn.Module 并重写 __init__ 和 forward 方法。
"""
class MyModule(nn.Module):
    def __init__(self):
        super().__init__() # 初始化继承自父类的属性
        # 卷积层 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 卷积层 2
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 展平层：将多维输入展平为一维
        self.flatten = nn.Flatten()

        # 全连接层
        self.linear1 = nn.Linear(64 * 7 * 7, 128)  # 64个7x7的特征图
        self.linear2 = nn.Linear(128, 10)  # 输出10类，对应与自己的分类任务有多少类

        # 输出层不加激活函数，损失函数CrossEntropyLoss会自动应用softmax

    def forward(self, x):
        # 卷积层 1
        x = self.pool1(self.relu1(self.conv1(x)))

        # 卷积层 2
        x = self.pool2(self.relu2(self.conv2(x)))

        # 全连接层
        x = self.relu1(self.linear1(self.flatten(x)))

        # 输出层
        x = self.linear2(x)

        return x

### 4，optim模块：优化模块，提供了多种常用的优化算法，用于更新模型的参数以最小化损失函数

In [1]:
from torch import optim

optimizer = optim.SGD(model.parameters(), lr=0.01) # SGD随机梯度下降

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) # Adam自适应调整每个参数的学习率，L2正则化

optimizer.zero_grad()  # 清空梯度

optimizer.step()  # 更新模型参数


NameError: name 'model' is not defined

### 5，autograd模块：用于自动求导（自动计算梯度）的核心模块

In [ ]:
# 不需要显式地导入。因为autograd是PyTorch中的内置功能，torch模块已经自动包含了autograd

loss.backward() # 反向传播

with torch.no_grad(): # 禁用自动求导

print(x.grad) # 打印x的梯度
